In [7]:
# Estimated running time: 1 second
import os
import network_generation as netGen
import RaCInG_input_generation as readIn
import numpy as np

Lmatrix, Rmatrix, Cdistr, LRdistr, cellTypes, ligs, recs, _ = readIn.generateInput(
    "min", "BLCA", folder_dir="/Users/joankant/Library/CloudStorage/OneDrive-UHN/RaCInG/output/BLCA")


In [8]:
print(Lmatrix.shape)
print(Rmatrix.shape)
print(Cdistr.shape)
print(LRdistr.shape)
print(cellTypes.shape)
print(ligs.shape)
print(recs.shape)


# Rmatrix, Cdistr[patient_id,:], LRdistr[:,:,patient_id]

(9, 276)
(9, 298)
(427, 9)
(276, 298, 427)
(9,)
(276,)
(298,)


In [9]:
np.random.seed(1)
# Note we use only data of patient 0 from the input data
patient_id = 0

N = 20
# 40  LR interactions -> avdeg = 2? 40/2
avdeg = 2 
V, E, types = netGen.model1(N, avdeg, Lmatrix, Rmatrix, Cdistr[patient_id,:], LRdistr[:,:,patient_id], genRandom = False)

In [10]:
print(V.shape)
print(E.shape)
print(types.shape)


(20,)
(35, 2)
(35, 2)


In [11]:
cellTypes

array(['B', 'CAF', 'CD8+ T', 'DC', 'Endo', 'M', 'NK', 'Treg', 'Tumor'],
      dtype='<U6')

In [12]:
print("The cell-types of each cell in the network we just generated are:")
print(cellTypes[V]) #Should be all tumor cells with the exception of cell 2 (B-cell) and cell 14 (endothelial cell).
print("The arc list of the network is:")
print(E) #Should start with a connection from cell 8 (a tumor cell) to cell 15 (a tumor cell).
print("The ligands of each connection:")
print(ligs[types[:,0]]) #The connection from cell 8 to 15 should have ligand L1CAM form cell 8.
print("The recepors of each connection:")
print(recs[types[:,1]]) #The connection from cell 8 to 15 should have receptor ERBB2 from cell 15.

The cell-types of each cell in the network we just generated are:
['DC' 'Treg' 'B' 'CD8+ T' 'CAF' 'B' 'CD8+ T' 'CD8+ T' 'CD8+ T' 'Endo' 'DC'
 'Treg' 'CD8+ T' 'Treg' 'B' 'Treg' 'DC' 'M' 'CAF' 'CD8+ T']
The arc list of the network is:
[[18 17]
 [11 17]
 [ 9  4]
 [18 17]
 [18 18]
 [13  7]
 [ 5 19]
 [11 12]
 [18 18]
 [ 6  0]
 [ 8 18]
 [ 0  6]
 [ 4  1]
 [18  9]
 [ 4  2]
 [ 1  0]
 [ 9  9]
 [ 9  4]
 [ 9 17]
 [ 4  9]
 [10 16]
 [ 4  9]
 [12 12]
 [15 17]
 [ 2  2]
 [17 17]
 [ 4  7]
 [15  0]
 [13  4]
 [17  4]
 [ 0 12]
 [10 18]
 [12  6]
 [ 0  3]
 [ 9  7]]
The ligands of each connection:
['VCAM1' 'CALR' 'BMP6' 'WNT5A' 'LUM' 'HLA-A' 'ICOSLG' 'CD58' 'WNT5A' 'APP'
 'ADAM17' 'SPP1' 'GRN' 'MDK' 'VEGFA' 'ADAM10' 'COL27A1' 'COL27A1' 'CCL5'
 'NFASC' 'BTLA' 'ADM2' 'MIF' 'OSM' 'ICAM1' 'ADAM10' 'MDK' 'ADAM15' 'TGFB1'
 'APOE' 'HLA-B' 'FAM3C' 'HLA-C' 'SEMA3C' 'ADM']
The recepors of each connection:
['ITGA4' 'ITGA3' 'ACVR2A' 'FZD5' 'ITGB1' 'APLP2' 'CTLA4' 'CD2' 'FZD2'
 'TNFRSF21' 'MET' 'ITGB1' 'TNFRSF1A' 'ITGA6' 

In [15]:
# Generate networks with a uniform LR-distribution. 
np.random.seed(1) # The same seed to compare with the previous output

# Making the LR-distribution uniform
def convert_to_uniform(LRdistr): 
    LRdistrUniform = np.zeros_like(LRdistr)
    normvec = 1 / np.count_nonzero(LRdistr, axis = (0,1)) 
    for i in range(LRdistr.shape[2]):
        copy = LRdistr[:,:,i].copy()
        copy[copy > 0] = normvec[i]
        LRdistrUniform[:,:,i] = copy
    return LRdistrUniform
LRdistrUniform = convert_to_uniform(LRdistr)
# Generating one graph instance with the uniform LR-dsitribution (20 nodes, and ~40 arcs)
Vnorm, Enorm, typesNorm = netGen.model1(N, avdeg, Lmatrix, Rmatrix, Cdistr[0,:], LRdistrUniform[:,:,0], genRandom = False)

print("The cells have exactly the same types as before:")
print(cellTypes[Vnorm]) # Same output as the previous code block
print("But the arcs have changed due to the new distribution:")
print(Enorm) # Starts with [8, 2]
print(LRdistr[:,:,0])

The cells have exactly the same types as before:
['DC' 'Treg' 'B' 'CD8+ T' 'CAF' 'B' 'CD8+ T' 'CD8+ T' 'CD8+ T' 'Endo' 'DC'
 'Treg' 'CD8+ T' 'Treg' 'B' 'Treg' 'DC' 'M' 'CAF' 'CD8+ T']
But the arcs have changed due to the new distribution:
[[ 4 12]
 [10  8]
 [ 9  3]
 [ 4 17]
 [ 4 10]
 [ 0 18]
 [12  3]
 [18  0]
 [13  5]
 [18  7]
 [ 9  0]
 [17  0]
 [ 9  0]
 [18 18]
 [ 1  9]
 [ 4  4]
 [ 4  3]
 [ 7 10]
 [ 4 18]
 [ 8 12]
 [10 10]
 [ 3  9]
 [ 8 16]
 [13  8]
 [ 6 12]
 [11 13]
 [11 13]
 [17 10]
 [ 3 13]
 [ 6  3]
 [ 4 17]
 [ 9  0]]
[[1.80110348e-03 1.53474587e-03 2.05366737e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.53474587e-03 2.05366737e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.0

In [19]:
Vnorm.shape, Enorm.shape, typesNorm.shape
np.savetxt("Vnorm.csv", Vnorm, delimiter=",")
np.savetxt("Enorm.csv", Enorm, delimiter=",")
np.savetxt("typesNorm.csv", typesNorm, delimiter=",")

In [29]:
Enorm

array([[ 4, 12],
       [10,  8],
       [ 9,  3],
       [ 4, 17],
       [ 4, 10],
       [ 0, 18],
       [12,  3],
       [18,  0],
       [13,  5],
       [18,  7],
       [ 9,  0],
       [17,  0],
       [ 9,  0],
       [18, 18],
       [ 1,  9],
       [ 4,  4],
       [ 4,  3],
       [ 7, 10],
       [ 4, 18],
       [ 8, 12],
       [10, 10],
       [ 3,  9],
       [ 8, 16],
       [13,  8],
       [ 6, 12],
       [11, 13],
       [11, 13],
       [17, 10],
       [ 3, 13],
       [ 6,  3],
       [ 4, 17],
       [ 9,  0]])

In [32]:
import Utilities as util
import pandas as pd
os.listdir(
    "/Users/joankant/Library/CloudStorage/OneDrive-UHN/RaCInG/output/40_cci_network_construction")

input_dir = "/Users/joankant/Library/CloudStorage/OneDrive-UHN/RaCInG/output/40_cci_network_construction/BLCA/20_2_norm"
V = pd.read_csv(f"{input_dir}/vertices.csv", header=None).values.flatten()
E = pd.read_csv(f"{input_dir}/edges.csv", header=None).values
types = pd.read_csv(f"{input_dir}/lr_info_edges.csv", header=None).values

# This function retains arcs from cells to itself
Adj = util.EdgetoAdj(E, len(V))
# This function removes arcs from cells to itself
AdjNoLoop = util.EdgetoAdj_No_loop(E, len(V))
print("The adjacency matrix with self loops is:")
# You see that "(2, 2) 2" appears in the third row. There are two self-loops from cell 2 to itself.
print(Adj)
print("The adjacency matrix without self loops is:")
print(AdjNoLoop)  # This is the same matrix as before, but with "(2, 2) 2" removed.


The adjacency matrix with self loops is:
  (0, 18)	1.0
  (1, 9)	1.0
  (3, 9)	1.0
  (3, 13)	1.0
  (4, 3)	1.0
  (4, 4)	1.0
  (4, 10)	1.0
  (4, 12)	1.0
  (4, 17)	2.0
  (4, 18)	1.0
  (6, 3)	1.0
  (6, 12)	1.0
  (7, 10)	1.0
  (8, 12)	1.0
  (8, 16)	1.0
  (9, 0)	3.0
  (9, 3)	1.0
  (10, 8)	1.0
  (10, 10)	1.0
  (11, 13)	2.0
  (12, 3)	1.0
  (13, 5)	1.0
  (13, 8)	1.0
  (17, 0)	1.0
  (17, 10)	1.0
  (18, 0)	1.0
  (18, 7)	1.0
  (18, 18)	1.0
The adjacency matrix without self loops is:
  (0, 18)	1.0
  (1, 9)	1.0
  (3, 9)	1.0
  (3, 13)	1.0
  (4, 3)	1.0
  (4, 10)	1.0
  (4, 12)	1.0
  (4, 17)	2.0
  (4, 18)	1.0
  (6, 3)	1.0
  (6, 12)	1.0
  (7, 10)	1.0
  (8, 12)	1.0
  (8, 16)	1.0
  (9, 0)	3.0
  (9, 3)	1.0
  (10, 8)	1.0
  (11, 13)	2.0
  (12, 3)	1.0
  (13, 5)	1.0
  (13, 8)	1.0
  (17, 0)	1.0
  (17, 10)	1.0
  (18, 0)	1.0
  (18, 7)	1.0


In [33]:
import feature_extraction as feats
NoTT, listTT = feats.Trust_Triangles(AdjNoLoop)
NoCT, listCT = feats.Cycle_Triangles(AdjNoLoop)
NoW, listW = feats.Wedges(AdjNoLoop)

print("Number of trust triangles: " + str(NoTT) + ". List of trust triangles:") #There should be 4
print(listTT) #Should start with [2, 12, 14]
print("Number of cycle triangles: " + str(NoCT) + ". List of cycle triangles:") #There should be 0
print(listCT) #Should only have the triangles []
print("Number of wedges: " + str(NoW) + ". List of wedges:") #There should be 27
print(listW) #Should start with [2, 9, 11]

Number of trust triangles: 3. List of trust triangles:
[[ 3  4 12]
 [10  4 17]
 [ 3  6 12]]
Number of cycle triangles: 0. List of cycle triangles:
[]
Number of wedges: 32. List of wedges:
[[ 0 18  7]
 [ 1  9  0]
 [ 1  9  3]
 [ 3  9  0]
 [ 3 13  5]
 [ 3 13  8]
 [ 4  3  9]
 [ 4  3 13]
 [ 4 10  8]
 [ 4 12  3]
 [ 4 17  0]
 [ 4 17 10]
 [ 4 18  0]
 [ 4 18  7]
 [ 6  3  9]
 [ 6  3 13]
 [ 6 12  3]
 [ 7 10  8]
 [ 8 12  3]
 [ 9  0 18]
 [ 9  3 13]
 [10  8 12]
 [10  8 16]
 [11 13  5]
 [11 13  8]
 [12  3  9]
 [12  3 13]
 [13  8 12]
 [13  8 16]
 [17  0 18]
 [17 10  8]
 [18  7 10]]
